In [1]:
from http.server import SimpleHTTPRequestHandler, HTTPServer
import urllib.parse
from urllib import request, error
import pandas as pd
from pycaret.classification import *
import sys
from http.server import BaseHTTPRequestHandler, HTTPServer
from requests import Session
import webbrowser
from cgi import parse_header, parse_multipart
from urllib.parse import parse_qs
from socketserver import ThreadingMixIn

In [2]:
sql_badwords = ['drop', 'select' ,'where' ,'from' ,'table', 'if (1=1) then', 'if (1=1) select', 'concat', ' char ', 'union', 'group by', 'having', 'order by', 'insert', 'exec', 'limit', 'waitfor', 'delay', 'sleep']
xss_badwords = ['script', 'alert', 'prompt', 'eval', 'onclick', 'onerror', 'onpropertychange', 'onresize', 'onload', 'onmouse', 'onblur', 'onkey', 'onfocus', 'fromCharCode', 'ontoggle', 'expression', 'onpointer']
ci_badwords = ['wget', 'etc', 'passwd', 'cmd', ' cat ', 'system', 'bin', 'curl', 'dir', 'echo', 'whoami', 'ifconfig', 'ipconfig', 'netsh', 'netstat', 'net use', 'perl', 'phpinfo', 'reg add']

#Feature extraction
def ExtractFeatures(path,body):
    sql_badwords_count = 0
    xss_badwords_count = 0
    ci_badwords_count = 0
    path = urllib.parse.unquote_plus(path)
    body = urllib.parse.unquote_plus(body)
    single_q = path.count("'") + body.count("'")
    double_q = path.count("\"") + body.count("\"")
    dashes = path.count("--") + body.count("--")
    braces = path.count("(") + body.count("(")
    spaces = path.count(" ") + body.count(" ")
    for word in sql_badwords:
        sql_badwords_count += path.count(word) + body.count(word)
    for word in xss_badwords:
        xss_badwords_count += path.count(word) + body.count(word)
    for word in ci_badwords:
        ci_badwords_count += path.count(word) + body.count(word)
    lst = [single_q,double_q,dashes,braces,spaces,sql_badwords_count,xss_badwords_count,ci_badwords_count]
    return pd.DataFrame([lst],columns=['single_q','double_q','dashes','braces','spaces','SQL Badwords','XSS Badwords','Command Injection Badords'])

In [3]:
Import Labelled Dataset
dataset = pd.read_csv(r'C:\Users\gmaster\Desktop\Diploma\Training dataset\Labelled dataset.csv')

In [4]:
exp_mclf101 = setup(data = dataset, target = 'class', session_id=123,numeric_features = ['XSS Badwords','SQL Badwords','Command Injection Badords','spaces','braces','single_q','dashes','double_q'],ignore_features = ['method', 'path', 'body'])

,Description,Value
0,session_id,123
1,Target,class
2,Target Type,Multiclass
3,Label Encoded,"Legit: 0, SQL Injection: 1, XSS: 2, command injection: 3"
4,Original Data,"(3791, 12)"
5,Missing Values,True
6,Numeric Features,8
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [5]:
et_model = create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9962,1.0000,0.9907,0.9963,0.9962,0.9948,0.9948
2,0.9962,0.9979,0.9966,0.9964,0.9963,0.9948,0.9948
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,0.9925,0.9971,0.9815,0.9926,0.9923,0.9894,0.9895
9,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [6]:
#tuned_et_model = tune_model(et_model)

In [7]:
final_et = finalize_model(et_model)

In [8]:
from http import client

#ML-WAF Development

class CallBackSrv(BaseHTTPRequestHandler):

  protocol_version = 'HTTP/1.1'
  baseurl = 'http://demo.testfire.net'
  session = Session()
  # Open a browser windows to our reverse proxy
  webbrowser.open('http://10.100.10.4:80/')

  def do_GET(self):
    resp = self.session.get(self.baseurl + self.path, allow_redirects=True)
    blank = " "
    live_data = ExtractFeatures(self.path,blank)
    result = predict_model(final_et, data = live_data)
    if result['Label'][0] == "SQL Injection":
        print("SQL Injection Detected")
        print("Malicious Request: " + self.path)
        print("Client IP: " + str(self.client_address))
        self.send_error(403, message="Request blocked by ML-WAF")
    elif result['Label'][0] == "XSS":
        print("XSS Attack Detected")
        print("Malicious Request: " + self.path)
        print("Client IP: " + str(self.client_address))
        self.send_error(403, message="Request blocked by ML-WAF")
    elif result['Label'][0] == "command injection":
        print("Command Injection Detected")
        print("Malicious Request: " + self.path)
        print("Client IP: " + str(self.client_address))
        self.send_error(403, message="Request blocked by ML-WAF")
    else:
        self.send_response(resp.status_code)
        self.send_header('Content-Length', len(resp.content))
        self.end_headers()
        self.wfile.write(resp.content)

        
  def parse_POST(self):
    ctype, pdict = parse_header(self.headers['content-type'])
    if ctype == 'multipart/form-data':
      postvars = parse_multipart(self.rfile, pdict)
    elif ctype == 'application/x-www-form-urlencoded':
      length = int(self.headers['content-length'])
      postvars = parse_qs(self.rfile.read(length),
                          keep_blank_values=1)
    else:
      postvars = {}
    return postvars

  def do_POST(self):
    length = int(self.headers.get('content-length'))
    field_data = self.rfile.read(length)
    fields = parse_qs(field_data)
    req_body = urllib.parse.urlencode(fields,doseq=True)
    live_data = ExtractFeatures(self.path,req_body)
    result = predict_model(final_et, data = live_data)
    if result['Label'][0] == "SQL Injection":
        print("SQL Injection Detected")
        print("Malicious Request: " + req_body)
        print("Client IP: " + str(self.client_address))
        self.send_error(403, message="Request blocked by ML-WAF")
    elif result['Label'][0] == "XSS":
        print("XSS Attack Detected")
        print("Malicious Request: " + req_body)
        print("Client IP: " + str(self.client_address))
        self.send_error(403, message="Request blocked by ML-WAF")
    elif result['Label'][0] == "command injection":
        print("Command Injection Detected")
        print("Malicious Request: " + req_body)
        print("Client IP: " + str(self.client_address))
        self.send_error(403, message="Request blocked by ML-WAF")
    else:
        postvars = self.parse_POST()
        resp = self.session.post(self.baseurl + self.path, data=postvars, allow_redirects=True)
        self.send_response(resp.status_code)
        self.send_header('Content-Length', len(resp.content))
        self.end_headers()
        self.wfile.write(resp.content)


class ThreadedHTTPServer(ThreadingMixIn, HTTPServer):
  """ Make our HTTP server multi-threaded """


httpd = ThreadedHTTPServer(('', 80), CallBackSrv)
httpd.serve_forever()

10.100.10.4 - - [23/Feb/2022 23:51:11] "GET / HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:12] "GET /style.css HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:12] "GET /images/home2.jpg HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:12] "GET /images/logo.gif HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:12] "GET /images/home1.jpg HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:12] "GET /images/pf_lock.gif HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:13] "GET /images/header_pic.jpg HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:13] "GET /images/home3.jpg HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:13] "GET /images/gradient.jpg HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:14] "GET /favicon.ico HTTP/1.1" 404 -
10.100.10.4 - - [23/Feb/2022 23:51:56] "GET /search.jsp?query=onmouseover%3D%22alert%281%29%22 HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:56] "GET /style.css HTTP/1.1" 200 -
10.100.10.4 - - [23/Feb/2022 23:51:57] "GET /images/heade

SQL Injection Detected
Malicious Request: /search.jsp?query=+select+*+from+users+where+id+%3D+1+or+%22%3F%5B%22+or+1+%3D+1+--+1
Client IP: ('10.100.10.4', 53841)


10.100.10.4 - - [23/Feb/2022 23:52:02] code 403, message Request blocked by ML-WAF
10.100.10.4 - - [23/Feb/2022 23:52:02] "GET /search.jsp?query=+select+*+from+users+where+id+%3D+1+or+%22%3F%5B%22+or+1+%3D+1+--+1 HTTP/1.1" 403 -
2.87.148.213 - - [24/Feb/2022 00:07:23] "GET / HTTP/1.1" 200 -
2.87.148.213 - - [24/Feb/2022 00:07:24] "GET /favicon.ico HTTP/1.1" 404 -
2.87.148.213 - - [24/Feb/2022 00:07:34] "GET /search.jsp?query=onmouseover+%3D+%27alert%27 HTTP/1.1" 200 -
10.100.10.4 - - [24/Feb/2022 00:08:00] "GET /login.jsp HTTP/1.1" 200 -
10.100.10.4 - - [24/Feb/2022 00:08:00] "GET /style.css HTTP/1.1" 200 -
10.100.10.4 - - [24/Feb/2022 00:08:00] "GET /images/logo.gif HTTP/1.1" 200 -
10.100.10.4 - - [24/Feb/2022 00:08:01] "GET /images/pf_lock.gif HTTP/1.1" 200 -
10.100.10.4 - - [24/Feb/2022 00:08:01] "GET /images/header_pic.jpg HTTP/1.1" 200 -
10.100.10.4 - - [24/Feb/2022 00:08:01] "GET /images/gradient.jpg HTTP/1.1" 200 -


XSS Attack Detected
Malicious Request: uid=onload%3Dalert%281%29&passw=sdsd&btnSubmit=Login
Client IP: ('10.100.10.4', 54029)


10.100.10.4 - - [24/Feb/2022 00:08:07] code 403, message Request blocked by ML-WAF
10.100.10.4 - - [24/Feb/2022 00:08:07] "POST /doLogin HTTP/1.1" 403 -
2.87.148.213 - - [24/Feb/2022 00:08:24] "GET /login.jsp HTTP/1.1" 200 -


XSS Attack Detected
Malicious Request: uid=onload%3Dalert%281%29&passw=sxdfsdf&btnSubmit=Login
Client IP: ('2.87.148.213', 2871)


2.87.148.213 - - [24/Feb/2022 00:08:30] code 403, message Request blocked by ML-WAF
2.87.148.213 - - [24/Feb/2022 00:08:30] "POST /doLogin HTTP/1.1" 403 -


KeyboardInterrupt: 